### 1. 문서를 읽는다

In [12]:
from docx import Document

doc=Document('./tax.docx')
full_text=''

for idx, para in enumerate(doc.paragraphs):
    full_text+=f'{para.text}\n'

### 2. 문서를 쪼갠다.

In [13]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder=tiktoken.encoding_for_model("gpt-4o")
    total_encoding=encoder.encode(full_text)
    total_token_count=len(total_encoding)
    text_list=[]
    
    for i in range(0,total_token_count,chunk_size):
        chunk=total_encoding[i:i+chunk_size]
        decoded=encoder.decode(chunk)
        text_list.append(decoded)
    
    return text_list

In [14]:
chunk_list=split_text(full_text,1500)

### 3. 문서 임베딩

In [15]:
import chromadb

chroma_client=chromadb.Client()

In [16]:
try:
    chroma_client.delete_collection("tax_collection")
except Exception:
    pass

In [17]:
collection_name='tax_collection'
#tax_collection=chroma_client.create_collection(collection_name)

In [25]:
# try:
#     chroma_client.delete_collection("tax_collection")
# except Exception:
#     pass

In [18]:
import os

from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY')
openai_embedding=OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [19]:
tax_collection=chroma_client.get_or_create_collection(collection_name,embedding_function=openai_embedding)

In [20]:
id_list=[]
for idx in range(len(chunk_list)):
    id_list.append(f'{idx}')

In [21]:
tax_collection.add(documents=chunk_list,ids=id_list)

### 4. 유사도 검색

In [22]:
query='연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc=tax_collection.query(query_texts=query, n_results=3)

### 5. LLM 질의

In [23]:
from openai import OpenAI

client=OpenAI()

response=client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role":"system","content":f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고하여 사용자의 질문에 답변해주세요. {retrieved_doc['documents'][0]}"},
        {"role":"user","content":query}
    ]
)

In [24]:
response.choices[0].message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 먼저 과세표준을 계산하고, 이에 따른 소득세율을 적용해야 합니다.\n\n1. **근로소득 공제**: 연간 근로소득에 따라 공제되는 금액입니다. 2023년 기준 공제율은 다음과 같습니다:\n   - 5천만원 이하는 550만원 + (연봉 - 1,500만원) * 40%\n\n   5천만원의 경우:\n   \\[\n   550만원 + (5,000만원 - 1,500만원) \\times 0.4 = 550만원 + 1,400만원 = 1,950만원\n   \\]\n\n2. **과세표준**: 총 연봉에서 근로소득 공제를 빼고 기본공제(인적공제 등) 등을 고려하여 계산합니다.\n   - 연봉 5천만원의 경우:\n   \\[\n   5,000만원 - 1,950만원 = 3,050만원\n   \\]\n   (여기서 추가 공제가 있을 경우 더 차감하여야 하지만 기본적인 계산만 진행합니다.)\n\n3. **소득세 계산**:\n   - 과세표준 3,050만원은 기본 세율 구간에 따라 계산합니다.\n   - 기본 세율 (2023년 기준):\n     - 1,200만원 이하: 6%\n     - 1,200만원 초과 ~ 4,600만원 이하: 15%\n   \n   \\[\n   (1,200만원 \\times 0.06) + (3,050만원 - 1,200만원) \\times 0.15 = 72만원 + 277.5만원 = 349.5만원\n   \\]\n\n4. **세액공제**: 근로소득 세액공제 및 기본 세액공제 등 해당 사항을 차감합니다.\n   - 근로소득 세액공제는 소득세의 일정 비율로 계산됩니다.\n   - 55만원 + (산출세액 × 30%) 입니다.\n   \\[\n   55만원 + (349.5만원 \\times 0.3) = 55만원 + 104.85만원 = 159.85만원\n   \\]\n\n5. **최종 소득세**:\n   - 산출 세액에서 세액공제를 차감:\n   \\[\n   349.5만원 - 159.85만원 = 189.65만원\n   